In [5]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import holidays

# For SARIMAX
from tqdm import tqdm
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# For FBProphet
from fbprophet import Prophet

# For LightGBM
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import optuna

In [8]:
def return_series(csv_path):
    """
    
    """
    
    def parse(x):
        return datetime.datetime.strptime(x, '%m/%d/%Y %H')
    
    df = pd.read_csv(csv_path, parse_dates=[['Date','Hour']], date_parser=parse)
    df = df.loc[~df.duplicated()]
    
    return df

In [9]:
df = return_series('mta.csv')

In [11]:
conditions = [
df['Plaza ID'].eq(21) | df['Plaza ID'].eq(1),
df['Plaza ID'].eq(22) | df['Plaza ID'].eq(2),
df['Plaza ID'].eq(23) | df['Plaza ID'].eq(3),
df['Plaza ID'].eq(24) | df['Plaza ID'].eq(4),
df['Plaza ID'].eq(25) | df['Plaza ID'].eq(5),
df['Plaza ID'].eq(26) | df['Plaza ID'].eq(6),
df['Plaza ID'].eq(27) | df['Plaza ID'].eq(7),
df['Plaza ID'].eq(28) | df['Plaza ID'].eq(8),
df['Plaza ID'].eq(29) | df['Plaza ID'].eq(9),
df['Plaza ID'].eq(30) | df['Plaza ID'].eq(11)
]
choices = [
    'Triboro',
    'Triboro',
    'Whitestone',
    'Henry Hudson',
    'Marine Parkway',
    'Cross Bay',
    'Queens Midtown',
    'Brooklyn Battery',
    'Throgs Neck',
    'Verrazano'
]
df['plaza_id'] = np.select(conditions, choices, 0)

In [14]:
df.rename(columns={'# Vehicles - E-ZPass': 'ezpass', '# Vehicles - VToll': 'cash'}, inplace=True)

In [15]:
df

,Date_Hour,Plaza ID,Direction,ezpass,cash,plaza_id
0,2020-10-24 00:00:00,21,I,1838,578,Triboro
1,2020-10-24 01:00:00,21,I,1084,372,Triboro
2,2020-10-24 02:00:00,21,I,805,260,Triboro
3,2020-10-24 03:00:00,21,I,766,211,Triboro
4,2020-10-24 04:00:00,21,I,1049,268,Triboro
...,...,...,...,...,...,...
1666242,2010-01-01 19:00:00,11,I,2675,1152,Verrazano
1666243,2010-01-01 20:00:00,11,I,2580,1086,Verrazano
1666244,2010-01-01 21:00:00,11,I,2302,987,Verrazano
1666245,2010-01-01 22:00:00,11,I,2170,910,Verrazano


In [16]:
df.loc[df['ezpass'].eq(0)]

,Date_Hour,Plaza ID,Direction,ezpass,cash,plaza_id
468119,2017-04-21 00:00:00,5,I,0,0,Marine Parkway
468120,2017-04-21 00:00:00,5,O,0,0,Marine Parkway
468121,2017-04-21 01:00:00,5,I,0,0,Marine Parkway
468123,2017-04-21 02:00:00,5,I,0,0,Marine Parkway
468431,2017-04-20 00:00:00,5,I,0,0,Marine Parkway
...,...,...,...,...,...,...
1633299,2010-03-14 02:00:00,8,I,0,0,Brooklyn Battery
1633300,2010-03-14 02:00:00,8,O,0,0,Brooklyn Battery
1633347,2010-03-14 02:00:00,9,I,0,0,Throgs Neck
1633348,2010-03-14 02:00:00,9,O,0,0,Throgs Neck


In [18]:
df.loc[df['cash'].eq(0)].describe()

,Plaza ID,ezpass,cash
count,7236.000000,7236.000000,7236.0
mean,7.256910,141.053759,0.0
std,5.203175,278.294733,0.0
min,1.000000,0.000000,0.0
25%,5.000000,0.000000,0.0
50%,6.000000,4.000000,0.0
75%,8.000000,180.250000,0.0
max,30.000000,4020.000000,0.0
